In [5]:
# app.py — FINAL DEPLOYABLE VERSION
!pip install streamlit plotly scikit-learn
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

st.set_page_config(page_title="Ad Profit AI by Bryan", layout="wide")
st.title("Ad Profit AI – Find & Fix Your Ad Money Leaks")
st.markdown("**Google Ads + LinkedIn + Meta → Instant profit insights & action plan**")

# ——— FILE UPLOADER (so clients can use their own data) ———
uploaded_files = st.file_uploader(
    "Upload your Google Ads, LinkedIn, and Meta CSV files",
    type=["csv"], accept_multiple_files=True
)

if uploaded_files:
    dfs = []
    for file in uploaded_files:
        df = pd.read_csv(file)
        if 'google' in file.name.lower():
            df['Platform'] = 'Google Ads'
        elif 'linkedin' in file.name.lower() or 'linked_in' in file.name.lower():
            df['Platform'] = 'LinkedIn'
        else:
            df['Platform'] = 'Meta'
        dfs.append(df)
    google, linkedin, meta = dfs[0], dfs[1], dfs[2] if len(dfs) == 3 else dfs[0], None, None
else:
    # Default data (your original files)
    google = pd.read_csv("/content/google_ads-sheet1-sourcetable.csv")
    linkedin = pd.read_csv("/content/linked_in_campaigns-sheet1-sourcetable.csv")
    meta = pd.read_csv("/content/meta_ads_campigns-sheet1-sourcetable.csv")

# ——— CLEAN & UNIFY (same as your working code) ———
def clean_and_unify(g, l, m):
    g['Cost'] = g['Cost'].replace(r'[$,]', '', regex=True).astype(float)
    l['Total Spent (USD)'] = l['Total Spent (USD)'].replace(r'[$,]', '', regex=True).astype(float)
    m['Amount spent'] = m['Amount spent'].replace(r'[$,]', '', regex=True).astype(float)
    if 'Purchase value' in m.columns:
        m['Purchase value'] = m['Purchase value'].replace(r'[$,]', '', regex=True).astype(float)

    g = g.rename(columns={'Day':'Date','Campaign':'Campaign Name','Country':'Region','Cost':'Spend'})
    g['Revenue'] = 0

    l = l.rename(columns={'Day':'Date','Total Spent (USD)':'Spend'})
    if 'Conversions' in l.columns: l.drop(columns=['Conversions'], inplace=True)
    l = l.rename(columns={'Leads':'Conversions'})
    l['Revenue'] = 0; l['Region'] = 'Global'; l['Device'] = 'Unknown'

    m = m.rename(columns={'Campaign name':'Campaign Name','Amount spent':'Spend','Purchases':'Conversions','Purchase value':'Revenue'})
    m['Region'] = 'Unknown'; m['Device'] = 'Mixed'

    cols = ['Date','Platform','Campaign Name','Region','Device','Impressions','Clicks','Spend','Conversions','Revenue']
    g = g.reindex(columns=cols, fill_value=0)
    l = l.reindex(columns=cols, fill_value=0)
    m = m.reindex(columns=cols, fill_value=0)

    df = pd.concat([g, l, m], ignore_index=True)
    df['ROAS'] = (df['Revenue']/df['Spend']).replace([np.inf, -np.inf], 0).fillna(0)
    return df

df = clean_and_unify(google, linkedin, meta)

# ML
X = df[['Spend','Conversions','Revenue','ROAS']].fillna(0)
X_scaled = StandardScaler().fit_transform(X)
df['Segment'] = KMeans(n_clusters=4, random_state=42, n_init=10).fit_predict(X_scaled)
df['Anomaly'] = IsolationForest(contamination=0.1).fit_predict(X_scaled)

# Dashboard
col1, col2, col3, col4 = st.columns(4)
col1.metric("Total Spend", f"${df['Spend'].sum():,.0f}")
col2.metric("Revenue", f"${df['Revenue'].sum():,.0f}")
col3.metric("Profit", f"${df['Revenue'].sum() - df['Spend'].sum():,.0f}")
col4.metric("ROAS", f"{df['ROAS'].mean():.2f}x")

bleeding = df[(df['Spend']>200) & (df['Conversions']==0) & (df['Revenue']==0)]
if len(bleeding)>0:
    st.error(f"PAUSE THESE {len(bleeding)} CAMPAIGNS → ${bleeding['Spend'].sum():,.0f} WASTED!")

st.success("Your Ad Profit AI is LIVE!")

fig = px.scatter(df, x="Spend", y="Revenue", color="Platform", size="ROAS", hover_data=["Campaign Name"])
st.plotly_chart(fig, width='stretch')

st.balloons()

2025-11-26 13:41:49.829 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 13:41:49.833 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 13:41:49.836 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 13:41:49.838 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 13:41:49.843 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 13:41:49.846 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 13:41:49.849 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 13:41:49.850 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()